# LangChain / LLMS - Solution

### ![](../img/installation-ico.png) Installation

In [ ]:
pip install langchain

### ![](../img/package-ico.png) Packages

In [ ]:
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.base import BaseCallbackHandler

### ![](../img/parametrage-ico.png) Paramétrages

In [ ]:
from typing import Union, Any, Dict
models = [ "llama2", "mistral", "codellama" ]

class MyHandler(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts, metadata, **kwargs
    ):
        print(f"\non_llm_start {metadata['name']}")
        
    def on_llm_end(
        self, response, **kwargs
    ):
        print(f"on_llm_end")

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], metadata, **kwargs: Any
    ) -> Any:
        print(f"on_chain_start {metadata['name']}")
        
    def on_chain_end(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], metadata, **kwargs: Any
    ) -> Any:
        print(f"on_chain_end {metadata['name']}")

llms = [ Ollama(model=m, callbacks=[MyHandler()], metadata={"name": m}) for m in models]
prompt = PromptTemplate.from_template(
    "Tell me a joke about {input}."
)
output_parser = StrOutputParser()

chains = [prompt | llm | output_parser for llm in llms]

### ![](../img/jouer-ico.png) Exécution(s)

In [ ]:
input = "elephant"
for c in chains:
    for s in c.invoke({"input": input}):
        print(s, end="", flush=True)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import ConfigurableField
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama().configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="llama2",
    # Ajout des autres options associées à d'autre clé
    mistral=ChatOllama(model="mistral"),
    codellama=ChatOllama(model="codellama"),
)

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""En tant qu'assistant IA, avant toute réponse indique absolument qui tu es et le nom de ton modèle.
                  Uniquement et seulement après, tu peux répondre à la demande de l'utilisateur
                  """),
    HumanMessage(content="Parle moi de LangChain.")
    ])
chain = prompt | llm

print(chain.invoke({}), end="\n")

configs = [{"llm": m} for m in ["mistral", "llama2", "codellama"]]
for c in configs:
    print(chain.with_config(configurable=c).invoke({}))
